In [1]:
import requests
import bs4
import csv
from tqdm import tqdm_notebook as tqdm
import json
import pprint

In [2]:
num=1
webpage= f"https://www.careerjet.it/wcerca/lavoro?s=analyst&l=Italia&lid=41162&b={num}"

In [3]:
print(webpage)

https://www.careerjet.it/wcerca/lavoro?s=analyst&l=Italia&lid=41162&b=1


In [4]:
response = requests.get(webpage)
print("Status: " + str(response.status_code))
doc = bs4.BeautifulSoup(response.text)

Status: 200


In [5]:
annunci= doc.find_all('div',class_="job")
len(annunci)
stringlen=doc.select("span.display_results")[0].text
ind=stringlen.find("su ") + 3
ind2= stringlen.find(" ",ind)
numannunci=int(doc.select("span.display_results")[0].text[ind:ind2])
ult= numannunci//len(annunci)+1
print(ult)

97


In [7]:
def parse_annuncio(annuncio):
    titolo = annuncio.select("a.title-company>p")[0].text
    _id= annuncio.select("a.title-company")[0]["id"]
    link= "https://www.careerjet.it" + prova[0].select("a.title-company")[0]["href"]
    try:
        azienda= annuncio.select("div.company_compact")[0].text
    except:
        azienda=""
    try:
        luogo= annuncio.select("div.locations_compact")[0].text
    except:
        luogo=""
    return {'_id': _id,'titolo': titolo, 'link': link, 'azienda': azienda, 'luogo': luogo}

In [8]:
annunci_list = []
for num in tqdm(range(1,(ult*20)+1,20)):
    webpage= f"https://www.careerjet.it/wcerca/lavoro?s=analyst&l=Italia&lid=41162&b={num}"
    response = requests.get(webpage)
    doc = bs4.BeautifulSoup(response.text)
    annunci= doc.find_all('div',class_="job")
    for annuncio in annunci:
        annunci_list.append(parse_annuncio(annuncio))
len(annunci_list)

IndexError: list index out of range

In [110]:
import pandas as pd
ds_annunci = pd.DataFrame(annunci_list)
ds_annunci.set_index("_id")
ds_annunci.head()

,_id,azienda,link,luogo,titolo
0,a_176c062cc9a6a797804c21e541279621,Randstad,https://www.careerjet.it/job/176c062cc9a6a7978...,\nMilano\n,Stage curricolare Sales Analyst
1,a_b6e4b1b085a30809373fe6edc18838f6,Difa Cooper S.p.A.,https://www.careerjet.it/job/176c062cc9a6a7978...,\nMilano\n,BUSINESS ANALYST
2,a_d1aea5ea5ef0794ba0055979842aad0c,Page Personnel,https://www.careerjet.it/job/176c062cc9a6a7978...,\nMilano\n,CATEGORIE PROTETTE - Business Analyst/Consultant
3,a_42dd69b095e34b78a89ab27193248008,Randstad,https://www.careerjet.it/job/176c062cc9a6a7978...,"\nFlero, Brescia\n",back office analyst
4,a_80fae76323cb3a0529602883846ce437,Randstad,https://www.careerjet.it/job/176c062cc9a6a7978...,\nBologna\n,Business Analyst


In [111]:
ds_annunci.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1755 entries, 0 to 1754
Data columns (total 5 columns):
_id        1755 non-null object
azienda    1755 non-null object
link       1755 non-null object
luogo      1755 non-null object
titolo     1755 non-null object
dtypes: object(5)
memory usage: 68.6+ KB


In [112]:
ds_annunci.to_csv("C:/Users/F013378/Desktop/analyst_annunci.csv")